In [0]:
# joins 
data1=[(1,'lokesh'),(2,'gopi'),(3,'hari'),(4,'ravi')]
schema1=['id','name']
df1=spark.createDataFrame(data1,schema1)
df1.show()
data2=[(1,'IT'),(2,'HR'),(3,'SALES'),(5,'SEC')]
schema2=['id','DEPT']
df2=spark.createDataFrame(data2,schema2)
df2.show()

+---+------+
| id|  name|
+---+------+
|  1|lokesh|
|  2|  gopi|
|  3|  hari|
|  4|  ravi|
+---+------+

+---+-----+
| id| DEPT|
+---+-----+
|  1|   IT|
|  2|   HR|
|  3|SALES|
|  5|  SEC|
+---+-----+



In [0]:
df1.join(df2,df1.id==df2.id,'inner').show()
df1.join(df2,df1.id==df2.id,'left').show()
df1.join(df2,df1.id==df2.id,'right').show()
df1.join(df2,df1.id==df2.id,'outer').show()



+---+------+---+-----+
| id|  name| id| DEPT|
+---+------+---+-----+
|  1|lokesh|  1|   IT|
|  2|  gopi|  2|   HR|
|  3|  hari|  3|SALES|
+---+------+---+-----+

+---+------+----+-----+
| id|  name|  id| DEPT|
+---+------+----+-----+
|  1|lokesh|   1|   IT|
|  2|  gopi|   2|   HR|
|  3|  hari|   3|SALES|
|  4|  ravi|null| null|
+---+------+----+-----+

+----+------+---+-----+
|  id|  name| id| DEPT|
+----+------+---+-----+
|   1|lokesh|  1|   IT|
|   2|  gopi|  2|   HR|
|   3|  hari|  3|SALES|
|null|  null|  5|  SEC|
+----+------+---+-----+

+----+------+----+-----+
|  id|  name|  id| DEPT|
+----+------+----+-----+
|   1|lokesh|   1|   IT|
|   2|  gopi|   2|   HR|
|   3|  hari|   3|SALES|
|   4|  ravi|null| null|
|null|  null|   5|  SEC|
+----+------+----+-----+



In [0]:
#left semi,left anti
# leftsami -> it is like inner join but gives you only matching rows those are only from left table
# leftanti -> it is like inner join but gives you only unmatched rows those are only from left table
# self - > used to join on dataframe itself
df1.join(df2,df1.id==df2.id,'leftsemi').show()
df1.join(df2,df1.id==df2.id,'leftanti').show()



+---+------+
| id|  name|
+---+------+
|  1|lokesh|
|  2|  gopi|
|  3|  hari|
+---+------+

+---+----+
| id|name|
+---+----+
|  4|ravi|
+---+----+



In [0]:
# self
data=[(1,'lokesh',0),(2,'gopi',1),(3,'hari',2),(4,'ravi',1)]
schema=['id','name','mgrid']
df1=spark.createDataFrame(data,schema)
df1.show()

+---+------+-----+
| id|  name|mgrid|
+---+------+-----+
|  1|lokesh|    0|
|  2|  gopi|    1|
|  3|  hari|    2|
|  4|  ravi|    1|
+---+------+-----+



In [0]:
from pyspark.sql.functions import col
df1.alias('emp').join(df1.alias('mgr'),col('emp.mgrid')==col('mgr.id'),'left').select(col('emp.name').alias('empname'),col('mgr.name').alias('mgrname')).show()


+-------+-------+
|empname|mgrname|
+-------+-------+
| lokesh|   null|
|   gopi| lokesh|
|   hari|   gopi|
|   ravi| lokesh|
+-------+-------+



In [0]:

data1=[(1,'lokesh','male','IT'),\
    (2,'gopi','male','SALES'),\
    (3,'hari','male','HR'),\
    (4,'ravi', 'male','IT'),\
    (5,'sai','female','IT'),\
    (6,'navi','female','HR')]
schema1=['id','name','gender','dept']
df1=spark.createDataFrame(data1,schema1)
df1.show()


+---+------+------+-----+
| id|  name|gender| dept|
+---+------+------+-----+
|  1|lokesh|  male|   IT|
|  2|  gopi|  male|SALES|
|  3|  hari|  male|   HR|
|  4|  ravi|  male|   IT|
|  5|   sai|female|   IT|
|  6|  navi|female|   HR|
+---+------+------+-----+



In [0]:
from pyspark.sql.functions import lit,col

In [0]:
# pivot -> it used to generate multiple columns of distinct values in a single column 
# it is a aggregte function we can also perform aggregate function on it
df1.groupBy('dept','gender').count().show()
df1.groupBy('dept').pivot('gender').count().show()


+-----+------+-----+
| dept|gender|count|
+-----+------+-----+
|   IT|  male|    2|
|SALES|  male|    1|
|   HR|  male|    1|
|   IT|female|    1|
|   HR|female|    1|
+-----+------+-----+

+-----+------+----+
| dept|female|male|
+-----+------+----+
|   HR|     1|   1|
|SALES|  null|   1|
|   IT|     1|   2|
+-----+------+----+



In [0]:
df2=df1.withColumn('dummy',lit('Dummy'))
display(df2)

id,name,gender,dept,dummy
1,lokesh,male,IT,Dummy
2,gopi,male,SALES,Dummy
3,hari,male,HR,Dummy
4,ravi,male,IT,Dummy
5,sai,female,IT,Dummy
6,navi,female,HR,Dummy


In [0]:
df2.groupBy('Dummy').pivot('dept').count().select(col('HR'),col('IT'),col('SALES')).show()

+---+---+-----+
| HR| IT|SALES|
+---+---+-----+
|  2|  3|    1|
+---+---+-----+



---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-212519517037227>:1
----> 1 help(pivot())

NameError: name 'pivot' is not defined

In [0]:
df=df1.groupBy('dept').pivot('gender').count().show()


+-----+------+----+
| dept|female|male|
+-----+------+----+
|   HR|     1|   1|
|SALES|  null|   1|
|   IT|     1|   2|
+-----+------+----+



In [0]:
# unppivot -> used to convert multiplecolumns into single column as rows 
# but in pyspark there is no unpivot function but we can achieve it by usung stack() function
from pyspark.sql.functions import expr
# in expr() we write code in string but it execute as a code

In [0]:
unpivotdf=df.select('dept',expr("stack(2,'M',male,'F',female) as ('gender',count)") )

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File <command-212519517037230>:1
----> 1 unpivotdf=df.select('dept',expr("stack(2,'M',male,'F',female) as ('gender',count)") )

AttributeError: 'NoneType' object has no attribute 'select'

In [0]:
data=[(1,'lokesh','male','IT'),\
    (2,'gopi','male',None),\
    (3,'hari','male','HR'),\
    (4,'ravi', 'male',None),\
    (5,'sai','female','IT'),\
    (6,'navi',None,'HR')]
schema=['id','name','gender','dept']
df=spark.createDataFrame(data,schema)
df.show()

+---+------+------+----+
| id|  name|gender|dept|
+---+------+------+----+
|  1|lokesh|  male|  IT|
|  2|  gopi|  male|null|
|  3|  hari|  male|  HR|
|  4|  ravi|  male|null|
|  5|   sai|female|  IT|
|  6|  navi|  null|  HR|
+---+------+------+----+



In [0]:
# fill & fillna used to replace your null values with values u want 
df.fillna('processing').show()

+---+------+----------+----------+
| id|  name|    gender|      dept|
+---+------+----------+----------+
|  1|lokesh|      male|        IT|
|  2|  gopi|      male|processing|
|  3|  hari|      male|        HR|
|  4|  ravi|      male|processing|
|  5|   sai|    female|        IT|
|  6|  navi|processing|        HR|
+---+------+----------+----------+



In [0]:
df.fillna('processing',['dept']).show()

+---+------+------+----------+
| id|  name|gender|      dept|
+---+------+------+----------+
|  1|lokesh|  male|        IT|
|  2|  gopi|  male|processing|
|  3|  hari|  male|        HR|
|  4|  ravi|  male|processing|
|  5|   sai|female|        IT|
|  6|  navi|  null|        HR|
+---+------+------+----------+



In [0]:
df.na.fill('pending',['gender']).show()

+---+------+-------+----+
| id|  name| gender|dept|
+---+------+-------+----+
|  1|lokesh|   male|  IT|
|  2|  gopi|   male|null|
|  3|  hari|   male|  HR|
|  4|  ravi|   male|null|
|  5|   sai| female|  IT|
|  6|  navi|pending|  HR|
+---+------+-------+----+

